# Integrate a regression tree model to classify a person's position based on landmark distances
- Collect Training Data: Capture images with known position labels (e.g., "Standing", "Push-up Up", "Push-up Down", "Lying Down").
- Extract Features: Compute landmark distances (e.g., shoulder-to-hip, hip-to-knee, elbow angle, etc.).
- Train a Decision Tree: Use scikit-learn's DecisionTreeClassifier or DecisionTreeRegressor to classify or predict positions.
- Integrate into Real-Time Detection: Use the trained model to classify the current pose.

---
# Embeddings in a CSV file

---

In [1]:
import cv2
import mediapipe as mp
import csv
import numpy as np
import os

2025-04-03 10:23:42.613090: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-04-03 10:23:42.630380: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-04-03 10:23:42.635442: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-04-03 10:23:42.646957: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
# Initialize MediaPipe Pose module
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5)

# Directory containing labeled folders with images
base_folder = "images"  # Change to your folder path

# Create CSV file and write headers
csv_filename = "landmark_positions.csv"
headers = ["shoulder_hip_dist", "hip_knee_dist", "elbow_angle", "label"]

with open(csv_filename, "w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow(headers)
    
    for label in os.listdir(base_folder):
        label_folder = os.path.join(base_folder, label)
        if not os.path.isdir(label_folder):
            continue
        
        for image_name in os.listdir(label_folder):
            image_path = os.path.join(label_folder, image_name)
            
            image = cv2.imread(image_path)
            if image is None:
                continue
            
            image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            results = pose.process(image_rgb)
            
            if results.pose_landmarks:
                landmarks = results.pose_landmarks.landmark
                
                # Get key landmarks
                shoulder = landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER]
                hip = landmarks[mp_pose.PoseLandmark.LEFT_HIP]
                knee = landmarks[mp_pose.PoseLandmark.LEFT_KNEE]
                elbow = landmarks[mp_pose.PoseLandmark.LEFT_ELBOW]
                wrist = landmarks[mp_pose.PoseLandmark.LEFT_WRIST]
                
                # Compute distances
                shoulder_hip_dist = np.linalg.norm([shoulder.x - hip.x, shoulder.y - hip.y])
                hip_knee_dist = np.linalg.norm([hip.x - knee.x, hip.y - knee.y])
                
                # Compute elbow angle
                def calculate_angle(a, b, c):
                    ba = np.array([a.x - b.x, a.y - b.y])
                    bc = np.array([c.x - b.x, c.y - b.y])
                    cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))
                    return np.degrees(np.arccos(np.clip(cosine_angle, -1.0, 1.0)))
                
                elbow_angle = calculate_angle(shoulder, elbow, wrist)
                
                # Save to CSV
                print( [shoulder_hip_dist, hip_knee_dist, elbow_angle, label] )
                writer.writerow([shoulder_hip_dist, hip_knee_dist, elbow_angle, label])
                print(f"Data saved: {shoulder_hip_dist}, {hip_knee_dist}, {elbow_angle}, {label}")
            
            cv2.imshow("Pose Capture", image)
            if cv2.waitKey(500) & 0xFF == ord('q'):
                break
    
cv2.destroyAllWindows()

I0000 00:00:1743668629.834339  659887 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1743668629.883385  677738 gl_context.cc:369] GL version: 3.2 (OpenGL ES 3.2 NVIDIA 550.120), renderer: NVIDIA GeForce GTX 1650/PCIe/SSE2
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
W0000 00:00:1743668630.008130  677730 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1743668630.060167  677736 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1743668630.079622  677737 landmark_projection_calculator.cc:186] Using NORM_RECT without IMAGE_DIMENSIONS is only supported for the square ROI. Provide IMAGE_DIMENSIONS or use PROJECTION_MATRIX.


[0.20033361208603676, 0.14085127522393512, 176.28145297060496, 'position_up']
Data saved: 0.20033361208603676, 0.14085127522393512, 176.28145297060496, position_up


QObject::moveToThread: Current thread (0x8f8ed70) is not the object's thread (0x9a74950).
Cannot move to target thread (0x8f8ed70)

QObject::moveToThread: Current thread (0x8f8ed70) is not the object's thread (0x9a74950).
Cannot move to target thread (0x8f8ed70)

QObject::moveToThread: Current thread (0x8f8ed70) is not the object's thread (0x9a74950).
Cannot move to target thread (0x8f8ed70)

QObject::moveToThread: Current thread (0x8f8ed70) is not the object's thread (0x9a74950).
Cannot move to target thread (0x8f8ed70)

QObject::moveToThread: Current thread (0x8f8ed70) is not the object's thread (0x9a74950).
Cannot move to target thread (0x8f8ed70)

QObject::moveToThread: Current thread (0x8f8ed70) is not the object's thread (0x9a74950).
Cannot move to target thread (0x8f8ed70)

QObject::moveToThread: Current thread (0x8f8ed70) is not the object's thread (0x9a74950).
Cannot move to target thread (0x8f8ed70)

QObject::moveToThread: Current thread (0x8f8ed70) is not the object's thread

[0.12727809340546978, 0.05259297517413317, 169.09024198876685, 'position_up']
Data saved: 0.12727809340546978, 0.05259297517413317, 169.09024198876685, position_up
[0.2667483134191944, 0.22173672975014505, 176.87449308757382, 'position_up']
Data saved: 0.2667483134191944, 0.22173672975014505, 176.87449308757382, position_up
[0.20679968648136068, 0.13016733032158403, 174.68305488281663, 'position_up']
Data saved: 0.20679968648136068, 0.13016733032158403, 174.68305488281663, position_up
[0.22169783213139016, 0.19175417819349694, 177.35954914202426, 'position_up']
Data saved: 0.22169783213139016, 0.19175417819349694, 177.35954914202426, position_up
[0.23492266173233856, 0.1730489651041111, 178.45397854539303, 'position_up']
Data saved: 0.23492266173233856, 0.1730489651041111, 178.45397854539303, position_up
[0.07594547179299335, 0.06874349077405148, 97.70851057384724, 'position_up']
Data saved: 0.07594547179299335, 0.06874349077405148, 97.70851057384724, position_up
[0.11766978589335945, 

---
# Classification

---

In [3]:
from sklearn.tree import DecisionTreeClassifier
import pandas as pd
import joblib

In [4]:
# Load collected data
data = pd.read_csv("landmark_positions.csv")  # Columns: distances + labels
# data = data.rename(columns={'pushups_up':'label', 'image_001.jpg':'name'})
# data = data.drop(columns=['name'])
data

,shoulder_hip_dist,hip_knee_dist,elbow_angle,label
0,0.200334,0.140851,176.281453,position_up
1,0.127278,0.052593,169.090242,position_up
2,0.266748,0.221737,176.874493,position_up
3,0.206800,0.130167,174.683055,position_up
4,0.221698,0.191754,177.359549,position_up
5,0.234923,0.173049,178.453979,position_up
6,0.075945,0.068743,97.708511,position_up
7,0.117670,0.128045,178.996235,position_up
8,0.259226,0.178172,154.779673,position_up
9,0.265918,0.183910,160.724634,position_up


In [5]:
X = data.drop(columns=['label'])  # Features: distances
y = data['label']  # Target: position labels

# Train classifier
clf_pushup = DecisionTreeClassifier()
clf_pushup.fit(X, y)

# Save model
joblib.dump(clf_pushup, "classifier.pkl")

['classifier.pkl']